### 증시자금동향

In [13]:
from cryptography.fernet import Fernet

# Fernet 키 생성
fernet_key = Fernet.generate_key()

print("생성된 Fernet 키:", fernet_key.decode())


생성된 Fernet 키: Egvzah0tWmBy-YFR0pNqEnzW2qNKAFPA5TCZHY2eppk=


In [22]:
xmltodict.__version__

'0.13.0'

In [1]:
import keyring
import requests as rq
from io import BytesIO
import zipfile
import xmltodict
import json
import pandas as pd
from sqlalchemy import create_engine
import pymysql
from datetime import date
from dateutil.relativedelta import relativedelta
from tqdm import tqdm
import time

In [10]:
pip show python-dateutil

Name: python-dateutil
Version: 2.8.2
Summary: Extensions to the standard Python datetime module
Home-page: https://github.com/dateutil/dateutil
Author: Gustavo Niemeyer
Author-email: gustavo@niemeyer.net
License: Dual License
Location: /opt/homebrew/Caskroom/miniconda/base/envs/yeardream/lib/python3.9/site-packages
Requires: six
Required-by: jupyter_client, matplotlib, pandas
Note: you may need to restart the kernel to use updated packages.


In [8]:
.__version__

NameError: name 'python' is not defined

In [48]:
import requests as rq
from bs4 import BeautifulSoup

url = 'https://finance.naver.com/sise/sise_deposit.naver'
data = rq.get(url)
data_html = BeautifulSoup(data.content)

In [49]:
parse_day = data_html.select_one(
    'div.subtop_sise_graph2 > ul.subtop_chart_note > li > span.tah').text

parse_day

'\xa0\xa0|\xa0\xa02023.09.19'

In [50]:
# 숫자에 해당하는 부분만 추출
import re

biz_day = re.findall('[0-9]+', parse_day)
biz_day = ''.join(biz_day)
biz_day

'20230919'

### 어떤 종목이 해당 국가에 상장되어 있는지에 대한 정보를 구해야함
-> 한국거래소 KRX (업종분류 현황, 개별종목 지표) 데이터 이용
- 업종분류 현황

In [51]:
# 업종분류 현황 크롤링

import requests as rq
from io import BytesIO
import pandas as pd

## 코스피 (stk)
gen_otp_url = 'http://data.krx.co.kr/comm/fileDn/GenerateOTP/generate.cmd'
gen_otp_stk = {
    'locale': 'ko_KR',
    'mktId': 'STK',
    'trdDd': biz_day,
    'money': '1',
    'csvxls_isNo': 'false',
    'name': 'fileDown',
    'url': 'dbms/MDC/STAT/standard/MDCSTAT03901'
}
# OTP 받기 전 나의 행적을 알려주기
headers = {'Referer': 'http://data.krx.co.kr/contents/MDC/MDI/mdiLoader'}

otp_stk = rq.post(gen_otp_url, gen_otp_stk, headers=headers).text
otp_stk # 이 부분(otp)을 url에 제출하면 데이터를 받을 수 있음.

down_url = 'http://data.krx.co.kr/comm/fileDn/download_csv/download.cmd'
down_sector_stk = rq.post(down_url, {'code': otp_stk}, headers=headers)
down_sector_stk.content # 받은 csv파일이 html형태로 나타남.

# 클랜징 처리 & 인코딩
sector_stk = pd.read_csv(BytesIO(down_sector_stk.content), encoding = 'EUC-KR')
sector_stk

,종목코드,종목명,시장구분,업종명,종가,대비,등락률,시가총액
0,095570,AJ네트웍스,KOSPI,서비스업,4460,140,3.24,208827435700
1,006840,AK홀딩스,KOSPI,기타금융,18640,-330,-1.74,246934537040
2,027410,BGF,KOSPI,기타금융,3625,5,0.14,346973367375
3,282330,BGF리테일,KOSPI,유통업,149200,-400,-0.27,2578758775200
4,138930,BNK금융지주,KOSPI,기타금융,7210,10,0.14,2322257637980
...,...,...,...,...,...,...,...,...
946,079980,휴비스,KOSPI,화학,6260,20,0.32,215970000000
947,005010,휴스틸,KOSPI,철강금속,5450,240,4.61,306225008750
948,000540,흥국화재,KOSPI,보험,3330,65,1.99,213928007850
949,000545,흥국화재우,KOSPI,보험,5540,10,0.18,4254720000


In [52]:
## 코스닥 (ksq)
gen_otp_url = 'http://data.krx.co.kr/comm/fileDn/GenerateOTP/generate.cmd'
gen_otp_ksq = {
    'locale': 'ko_KR',
    'mktId': 'KSQ',
    'trdDd': biz_day,
    'money': '1',
    'csvxls_isNo': 'false',
    'name': 'fileDown',
    'url': 'dbms/MDC/STAT/standard/MDCSTAT03901'
}
# OTP 받기 전 나의 행적을 알려줘야 로봇으로 인식하지 않음.
headers = {'Referer': 'http://data.krx.co.kr/contents/MDC/MDI/mdiLoader'}

otp_ksq = rq.post(gen_otp_url, gen_otp_ksq, headers=headers).text
otp_ksq # 이 부분(otp)을 url에 제출하면 데이터를 받을 수 있음.

down_url = 'http://data.krx.co.kr/comm/fileDn/download_csv/download.cmd'
down_sector_ksq = rq.post(down_url, {'code': otp_ksq}, headers=headers)
down_sector_ksq.content # 받은 csv파일이 html형태로 나타남.

# 이를 클랜징 처리 & 인코딩
sector_ksq = pd.read_csv(BytesIO(down_sector_ksq.content), encoding = 'EUC-KR')
sector_ksq

,종목코드,종목명,시장구분,업종명,종가,대비,등락률,시가총액
0,060310,3S,KOSDAQ,기계·장비,2580,210,8.86,125224536360
1,054620,APS,KOSDAQ,금융,8140,-90,-1.09,166008958940
2,265520,AP시스템,KOSDAQ,반도체,17950,-50,-0.28,274301506950
3,211270,AP위성,KOSDAQ,통신장비,15500,20,0.13,233775712000
4,126600,BGF에코머티리얼즈,KOSDAQ,화학,4820,-65,-1.33,260440385500
...,...,...,...,...,...,...,...,...
1669,024060,흥구석유,KOSDAQ,유통,8210,110,1.36,123150000000
1670,010240,흥국,KOSDAQ,기계·장비,6630,200,3.11,81699474480
1671,189980,흥국에프엔비,KOSDAQ,음식료·담배,2330,-25,-1.06,93521136910
1672,037440,희림,KOSDAQ,기타서비스,9250,680,7.93,128782893750


In [53]:
### 두 종목을 합침. (concat)
krx_sector = pd.concat([sector_stk, sector_ksq]).reset_index(drop = True)
krx_sector

,종목코드,종목명,시장구분,업종명,종가,대비,등락률,시가총액
0,095570,AJ네트웍스,KOSPI,서비스업,4460,140,3.24,208827435700
1,006840,AK홀딩스,KOSPI,기타금융,18640,-330,-1.74,246934537040
2,027410,BGF,KOSPI,기타금융,3625,5,0.14,346973367375
3,282330,BGF리테일,KOSPI,유통업,149200,-400,-0.27,2578758775200
4,138930,BNK금융지주,KOSPI,기타금융,7210,10,0.14,2322257637980
...,...,...,...,...,...,...,...,...
2620,024060,흥구석유,KOSDAQ,유통,8210,110,1.36,123150000000
2621,010240,흥국,KOSDAQ,기계·장비,6630,200,3.11,81699474480
2622,189980,흥국에프엔비,KOSDAQ,음식료·담배,2330,-25,-1.06,93521136910
2623,037440,희림,KOSDAQ,기타서비스,9250,680,7.93,128782893750


In [54]:
#### 종목명 null 데이터 클랜징 (strip) & 기준일 추가.
krx_sector['종목명'] = krx_sector['종목명'].str.strip()
krx_sector['기준일'] = biz_day

krx_sector

,종목코드,종목명,시장구분,업종명,종가,대비,등락률,시가총액,기준일
0,095570,AJ네트웍스,KOSPI,서비스업,4460,140,3.24,208827435700,20230919
1,006840,AK홀딩스,KOSPI,기타금융,18640,-330,-1.74,246934537040,20230919
2,027410,BGF,KOSPI,기타금융,3625,5,0.14,346973367375,20230919
3,282330,BGF리테일,KOSPI,유통업,149200,-400,-0.27,2578758775200,20230919
4,138930,BNK금융지주,KOSPI,기타금융,7210,10,0.14,2322257637980,20230919
...,...,...,...,...,...,...,...,...,...
2620,024060,흥구석유,KOSDAQ,유통,8210,110,1.36,123150000000,20230919
2621,010240,흥국,KOSDAQ,기계·장비,6630,200,3.11,81699474480,20230919
2622,189980,흥국에프엔비,KOSDAQ,음식료·담배,2330,-25,-1.06,93521136910,20230919
2623,037440,희림,KOSDAQ,기타서비스,9250,680,7.93,128782893750,20230919


- 개별종목 지표 (배당수익률)

In [55]:
# 개별종목 지표 크롤링
gen_otp_url = 'http://data.krx.co.kr/comm/fileDn/GenerateOTP/generate.cmd'
gen_otp_data = {
    'locale': 'ko_KR',
    'searchType': '1',
    'mktId': 'ALL',
    'trdDd': biz_day,
    'csvxls_isNo': 'false',
    'name': 'fileDown',
    'url': 'dbms/MDC/STAT/standard/MDCSTAT03501'
}
headers = {'Referer': 'http://data.krx.co.kr/contents/MDC/MDI/mdiLoader'}
otp = rq.post(gen_otp_url, gen_otp_data, headers=headers).text

down_url = 'http://data.krx.co.kr/comm/fileDn/download_csv/download.cmd'
krx_ind = rq.post(down_url, {'code': otp}, headers=headers)

krx_ind = pd.read_csv(BytesIO(krx_ind.content), encoding='EUC-KR')
krx_ind['종목명'] = krx_ind['종목명'].str.strip()
krx_ind['기준일'] = biz_day

krx_ind

,종목코드,종목명,종가,대비,등락률,EPS,PER,선행 EPS,선행 PER,BPS,PBR,주당배당금,배당수익률,기준일
0,060310,3S,2580,210,8.86,30.0,86.00,NaN,NaN,947.0,2.72,0,0.00,20230919
1,095570,AJ네트웍스,4460,140,3.24,201.0,22.19,593.0,7.51,8076.0,0.55,270,6.05,20230919
2,006840,AK홀딩스,18640,-330,-1.74,NaN,NaN,NaN,NaN,41948.0,0.44,200,1.07,20230919
3,054620,APS,8140,-90,-1.09,505.0,16.12,NaN,NaN,10864.0,0.75,0,0.00,20230919
4,265520,AP시스템,17950,-50,-0.28,5463.0,3.29,NaN,NaN,17980.0,1.00,270,1.50,20230919
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2570,000540,흥국화재,3330,65,1.99,2142.0,1.55,NaN,NaN,11208.0,0.30,0,0.00,20230919
2571,000545,흥국화재우,5540,10,0.18,NaN,NaN,NaN,NaN,NaN,NaN,0,0.00,20230919
2572,003280,흥아해운,1743,-7,-0.40,94.0,18.54,NaN,NaN,544.0,3.20,0,0.00,20230919
2573,037440,희림,9250,680,7.93,567.0,16.31,NaN,NaN,5186.0,1.78,150,1.62,20230919


### 두 데이터를 합치기

In [56]:
# 중복되지 않는 하나의 지표에만 존재하는 종목명 데이터 체크
set(krx_sector['종목명']).symmetric_difference(set(krx_ind['종목명']))

{'ESR켄달스퀘어리츠',
 'GRT',
 'JTC',
 'KB스타리츠',
 'NH올원리츠',
 'NH프라임리츠',
 'SBI핀테크솔루션즈',
 'SK리츠',
 '골든센츄리',
 '글로벌에스엠',
 '네오이뮨텍',
 '디앤디플랫폼리츠',
 '로스웰',
 '롯데리츠',
 '마스턴프리미어리츠',
 '맥쿼리인프라',
 '맵스리얼티1',
 '메디포스트',
 '메디포스트 [락]',
 '모두투어리츠',
 '미래에셋글로벌리츠',
 '미래에셋맵스리츠',
 '미투젠',
 '바다로19호',
 '삼성FN리츠',
 '소마젠',
 '신한서부티엔디리츠',
 '신한알파리츠',
 '씨케이에이치',
 '애머릿지',
 '에이리츠',
 '엑세스바이오',
 '엘브이엠씨홀딩스',
 '오가닉티코스메틱',
 '윙입푸드',
 '이리츠코크렙',
 '이스트아시아홀딩스',
 '이지스레지던스리츠',
 '이지스밸류리츠',
 '잉글우드랩',
 '제이알글로벌리츠',
 '컬러레이',
 '케이탑리츠',
 '코람코더원리츠',
 '코람코에너지리츠',
 '코오롱티슈진',
 '크리스탈신소재',
 '프레스티지바이오파마',
 '한국ANKOR유전',
 '한국패러랠',
 '한화리츠',
 '헝셩그룹'}

In [57]:
# 두 데이터를 합쳐주자.
kor_ticker = pd.merge(krx_sector,
                      krx_ind,
                      on=krx_sector.columns.intersection(krx_ind.columns).tolist(),
                      how='outer')

kor_ticker

,종목코드,종목명,시장구분,업종명,종가,대비,등락률,시가총액,기준일,EPS,PER,선행 EPS,선행 PER,BPS,PBR,주당배당금,배당수익률
0,095570,AJ네트웍스,KOSPI,서비스업,4460,140,3.24,2.088274e+11,20230919,201.0,22.19,593.0,7.51,8076.0,0.55,270.0,6.05
1,006840,AK홀딩스,KOSPI,기타금융,18640,-330,-1.74,2.469345e+11,20230919,NaN,NaN,NaN,NaN,41948.0,0.44,200.0,1.07
2,027410,BGF,KOSPI,기타금융,3625,5,0.14,3.469734e+11,20230919,247.0,14.68,650.0,5.57,16528.0,0.22,110.0,3.03
3,282330,BGF리테일,KOSPI,유통업,149200,-400,-0.27,2.578759e+12,20230919,11203.0,13.32,13573.0,10.99,55724.0,2.68,4100.0,2.75
4,138930,BNK금융지주,KOSPI,기타금융,7210,10,0.14,2.322258e+12,20230919,2404.0,3.00,2553.0,2.82,30468.0,0.24,625.0,8.67
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2621,010240,흥국,KOSDAQ,기계·장비,6630,200,3.11,8.169947e+10,20230919,740.0,8.96,NaN,NaN,7971.0,0.83,220.0,3.32
2622,189980,흥국에프엔비,KOSDAQ,음식료·담배,2330,-25,-1.06,9.352114e+10,20230919,309.0,7.54,250.0,9.31,2295.0,1.02,40.0,1.72
2623,037440,희림,KOSDAQ,기타서비스,9250,680,7.93,1.287829e+11,20230919,567.0,16.31,NaN,NaN,5186.0,1.78,150.0,1.62
2624,238490,힘스,KOSDAQ,반도체,6370,-30,-0.47,7.205894e+10,20230919,NaN,NaN,NaN,NaN,5656.0,1.13,0.0,0.00


In [58]:
# 종목구분 만들어주기

import numpy as np

diff = list(set(krx_sector['종목명']).symmetric_difference(set(krx_ind['종목명'])))

kor_ticker['종목구분'] = np.where(kor_ticker['종목명'].str.contains('스팩|제[0-9]+호'), '스팩',
                              np.where(kor_ticker['종목코드'].str[-1:] != '0', '우선주',
                                       np.where(kor_ticker['종목명'].str.endswith('리츠'), '리츠',
                                                np.where(kor_ticker['종목명'].isin(diff), '기타',
                                                         '보통주'))))

kor_ticker['종목구분'].value_counts()

종목구분
보통주    2373
우선주     118
스팩       83
기타       30
리츠       22
Name: count, dtype: int64

In [59]:
# 추가적인 클랜징처리.

kor_ticker = kor_ticker.reset_index(drop=True)
kor_ticker.columns = kor_ticker.columns.str.replace(' ', '') # 컬럼명 공백 제거
kor_ticker = kor_ticker[['종목코드', '종목명', '시장구분', '종가', '시가총액', '기준일', 'EPS', '선행EPS', 'BPS', '주당배당금', '종목구분']] # 원하는 것만 선택
kor_ticker = kor_ticker.replace({np.nan: None}) # nan은 SQL에 저장할 수 없으므로 None으로 변경

In [60]:
kor_ticker

,종목코드,종목명,시장구분,종가,시가총액,기준일,EPS,선행EPS,BPS,주당배당금,종목구분
0,095570,AJ네트웍스,KOSPI,4460,208827435700.0,20230919,201.0,593.0,8076.0,270.0,보통주
1,006840,AK홀딩스,KOSPI,18640,246934537040.0,20230919,None,None,41948.0,200.0,보통주
2,027410,BGF,KOSPI,3625,346973367375.0,20230919,247.0,650.0,16528.0,110.0,보통주
3,282330,BGF리테일,KOSPI,149200,2578758775200.0,20230919,11203.0,13573.0,55724.0,4100.0,보통주
4,138930,BNK금융지주,KOSPI,7210,2322257637980.0,20230919,2404.0,2553.0,30468.0,625.0,보통주
...,...,...,...,...,...,...,...,...,...,...,...
2621,010240,흥국,KOSDAQ,6630,81699474480.0,20230919,740.0,None,7971.0,220.0,보통주
2622,189980,흥국에프엔비,KOSDAQ,2330,93521136910.0,20230919,309.0,250.0,2295.0,40.0,보통주
2623,037440,희림,KOSDAQ,9250,128782893750.0,20230919,567.0,None,5186.0,150.0,보통주
2624,238490,힘스,KOSDAQ,6370,72058943320.0,20230919,None,None,5656.0,0.0,보통주


> 이제 이 데이터를 sql에 저장. (20:44)

### 섹터정보 크롤링

In [30]:
import json
import requests as rq
import pandas as pd

url = f'''https://www.wiseindex.com/Index/GetIndexComponets?ceil_yn=0&dt={biz_day}&sec_cd=G10'''
data = rq.get(url).json()

In [31]:
# 데이터프레임
data_pd = pd.json_normalize(data['list'])
data_pd

,IDX_CD,IDX_NM_KOR,ALL_MKT_VAL,CMP_CD,CMP_KOR,MKT_VAL,WGT,S_WGT,CAL_WGT,SEC_CD,SEC_NM_KOR,SEQ,TOP60,APT_SHR_CNT
0,G10,WICS 에너지,25263157,096770,SK이노베이션,9246962,36.60,36.60,1.0,G10,에너지,1,3,56315236
1,G10,WICS 에너지,25263157,009830,한화솔루션,3465354,13.72,50.32,1.0,G10,에너지,2,3,108292298
2,G10,WICS 에너지,25263157,010950,S-Oil,3328285,13.17,63.49,1.0,G10,에너지,3,3,41655633
3,G10,WICS 에너지,25263157,267250,HD현대,3100953,12.27,75.77,1.0,G10,에너지,4,3,44236128
4,G10,WICS 에너지,25263157,078930,GS,2006740,7.94,83.71,1.0,G10,에너지,5,3,49245150
5,G10,WICS 에너지,25263157,112610,씨에스윈드,1372089,5.43,89.14,1.0,G10,에너지,6,3,23615986
6,G10,WICS 에너지,25263157,006120,SK디스커버리,443439,1.76,90.90,1.0,G10,에너지,7,3,10470820
7,G10,WICS 에너지,25263157,270520,지오릿에너지,246499,0.98,91.87,1.0,G10,에너지,8,3,23387003
8,G10,WICS 에너지,25263157,005090,SGC에너지,211217,0.84,92.71,1.0,G10,에너지,9,3,7925582
9,G10,WICS 에너지,25263157,044490,태웅,191431,0.76,93.47,1.0,G10,에너지,10,3,9203395


In [32]:
# 섹터정보 크롤링

sector_code = [
    'G25', 'G35', 'G50', 'G40', 'G10', 'G20', 'G55', 'G30', 'G15', 'G45'
]

data_sector = []

for i in tqdm(sector_code):
    url = f'''https://www.wiseindex.com/Index/GetIndexComponets?ceil_yn=0&dt={biz_day}&sec_cd={i}'''
    data = rq.get(url).json()
    data_pd = pd.json_normalize(data['list'])
    
    data_sector.append(data_pd)
    time.sleep(1)

100%|██████████| 10/10 [00:12<00:00,  1.27s/it]


In [33]:
kor_sector = pd.concat(data_sector, axis = 0)
kor_sector
kor_sector = kor_sector[['IDX_CD', 'CMP_CD', 'CMP_KOR', 'SEC_NM_KOR']]
kor_sector['기준일'] = biz_day
kor_sector['기준일'] = pd.to_datetime(kor_sector['기준일'])
kor_sector

,IDX_CD,CMP_CD,CMP_KOR,SEC_NM_KOR,기준일
0,G25,005380,현대차,경기관련소비재,2023-09-19
1,G25,000270,기아,경기관련소비재,2023-09-19
2,G25,012330,현대모비스,경기관련소비재,2023-09-19
3,G25,051900,LG생활건강,경기관련소비재,2023-09-19
4,G25,090430,아모레퍼시픽,경기관련소비재,2023-09-19
...,...,...,...,...,...
648,G45,032960,동일기연,IT,2023-09-19
649,G45,067730,로지시스,IT,2023-09-19
650,G45,372800,아이티아이즈,IT,2023-09-19
651,G45,067770,세진티에스,IT,2023-09-19


> 티커랑 마찬가지로 sql에 저장 (30:42)

In [34]:
kor_ticker.to_csv('kor_ticker.csv', encoding='EUC-KR')

In [35]:
kor_sector.to_csv('kor_sector.csv', encoding='EUC-KR')

In [67]:
ip = '43.201.116.164'
engine = create_engine(f'mysql+pymysql://jun:12345678@{ip}:3306/quant')
con = pymysql.connect(user='jun',
                      passwd='12345678',
                      host=f'{ip}',
                      db='quant',
                      charset='utf8')
mycursor = con.cursor()


In [69]:
kor_ticker

,종목코드,종목명,시장구분,종가,시가총액,기준일,EPS,선행EPS,BPS,주당배당금,종목구분
0,095570,AJ네트웍스,KOSPI,4460,208827435700.0,20230919,201.0,593.0,8076.0,270.0,보통주
1,006840,AK홀딩스,KOSPI,18640,246934537040.0,20230919,None,None,41948.0,200.0,보통주
2,027410,BGF,KOSPI,3625,346973367375.0,20230919,247.0,650.0,16528.0,110.0,보통주
3,282330,BGF리테일,KOSPI,149200,2578758775200.0,20230919,11203.0,13573.0,55724.0,4100.0,보통주
4,138930,BNK금융지주,KOSPI,7210,2322257637980.0,20230919,2404.0,2553.0,30468.0,625.0,보통주
...,...,...,...,...,...,...,...,...,...,...,...
2621,010240,흥국,KOSDAQ,6630,81699474480.0,20230919,740.0,None,7971.0,220.0,보통주
2622,189980,흥국에프엔비,KOSDAQ,2330,93521136910.0,20230919,309.0,250.0,2295.0,40.0,보통주
2623,037440,희림,KOSDAQ,9250,128782893750.0,20230919,567.0,None,5186.0,150.0,보통주
2624,238490,힘스,KOSDAQ,6370,72058943320.0,20230919,None,None,5656.0,0.0,보통주


In [70]:
kor_ticker.to_sql(name='kor_ticker', con=engine, index=True, if_exists='append')

2626

In [71]:
ticker_list = pd.read_sql("""
                          select * from kor_ticker
                          where 기준일 = (select max(기준일) from ticker) and 종목구분 = '보통주';
                          """, con=engine)

ProgrammingError: (pymysql.err.ProgrammingError) (1146, "Table 'quant.ticker' doesn't exist")
[SQL: 
                          select * from kor_ticker
                          where 기준일 = (select max(기준일) from ticker) and 종목구분 = '보통주';
                          ]
(Background on this error at: https://sqlalche.me/e/14/f405)

In [37]:
# kor_sector.to_sql(name='sector', con=engine, index=True, if_exists='append')